<a href="https://colab.research.google.com/github/bob734243/FreeCodeCamp_Machine-Learning-with-Python/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:


# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Look at the data
dataset.describe()

In [ ]:
# Shape
dataset.shape

In [ ]:
# One-hot encoding for 'sex', 'smoker', 'region'
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset in training and test
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Pop off expenses
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# Normalize data
norm = keras.layers.Normalization()
norm.adapt(train_dataset.to_numpy())

# Define model
model = keras.Sequential([
    norm,
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, input_shape=[train_dataset.shape[1]])
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

In [ ]:
# Fit the model
model.fit(
    train_dataset, train_labels,
    epochs=100,
    validation_data=(test_dataset, test_labels),
    verbose=1
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
